In [29]:
import sys
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import optuna
import statistics
import re
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectFromModel, SequentialFeatureSelector
from functools import partial
import warnings

warnings.filterwarnings('ignore')


In [30]:
sys.path.append(os.path.abspath("/home/bk_anupam/code/ML/ML_UTILS/"))

In [31]:
import train_tabular_utils as tt
import cv_split_utils
import enums
import data_utils

In [32]:
RUN_MODE = "LOCAL"
USE_ORIGINAL_DATA = True
DATA_READPATH = "./data/"
DATA_WRITEPATH = "./data/"
if RUN_MODE == "KAGGLE":    
    DATA_READPATH = "/kaggle/input/playground-series-s4e6/"
    DATA_WRITEPATH = "/kaggle/working/"

In [33]:
# import train dataset locally from data folder
df_train = pd.read_csv(DATA_READPATH + "train.csv")
# import test dataset locally from data folder
df_test = pd.read_csv(DATA_READPATH + "test.csv")
df_train.drop(columns=['id'], axis=1, inplace=True)
df_test.drop(columns=['id'], axis=1, inplace=True)
print(f"df_train shape: {df_train.shape}")
print(f"df_test shape: {df_test.shape}")

df_train shape: (188533, 12)
df_test shape: (125690, 11)


In [34]:
if USE_ORIGINAL_DATA:
    df_train_orig = pd.read_csv("./data/used_cars.csv")
    df_train_orig[['milage', 'price']] = df_train_orig[['milage', 'price']].applymap(lambda x: int(re.sub("[^0-9]", "", x)))
    df_train_orig['milage'] = df_train_orig['milage'].astype('int64')
    df_train_orig['price'] = df_train_orig['price'].astype('int64')
    # add df_train_orig rows to df_train
    df_train = pd.concat([df_train, df_train_orig], axis=0, ignore_index=True)
    print(f"df_train shape: {df_train.shape}")

df_train shape: (192542, 12)


In [35]:
def extract_engine_info(engine_desc: str, brand: str) -> dict:
    # check if engine_desc is a valid string
    if not isinstance(engine_desc, str):
        return None
    engine_desc = engine_desc.lower()
    brand = brand.lower()
    # Define patterns for each attribute
    horsepower_pattern = r'(\d+(\.\d+)?\s*)hp'
    capacity_pattern = r'(\d+(\.\d+)?\s*)l'
    cylinders_pattern = r'(\d+)\s*cylinder|v(\d+)'
    fuel_pattern = r'(gasoline|diesel|flex|electric|dohc|ohv)'    
    # Extract horsepower
    horsepower_match = re.search(horsepower_pattern, engine_desc)
    horsepower = float(horsepower_match.group(1)) if horsepower_match else None
    # Extract capacity
    capacity_match = re.search(capacity_pattern, engine_desc)
    capacity = float(capacity_match.group(1)) if capacity_match else None
    # Extract cylinders
    cylinders_match = re.search(cylinders_pattern, engine_desc)    
    cylinders = int(cylinders_match.group(1) or cylinders_match.group(2)) if cylinders_match else None
    # Extract fuel type
    fuel_match = re.search(fuel_pattern, engine_desc)
    fuel = fuel_match.group(0) if fuel_match else None
    if fuel in ('dohc', 'ohv'):
        fuel = "gasoline"
    # check is fuel_type is None and brand is "Tesla" then set fuel_type to "electric"
    if fuel is None and brand == "tesla":
        fuel = "electric"
    # Extract turbo
    turbo_match = re.search(r'turbo', engine_desc)
    turbo = True if turbo_match else False
    # extract hybrid
    hybrid_match = re.search(r'hybrid', engine_desc)
    hybrid = True if hybrid_match else False
    return {
        "horsepower": horsepower, 
        "capacity": capacity,
        "cylinders": cylinders,
        "fuel": fuel,
        "turbo": turbo,
        "hybrid": hybrid
    }

In [36]:
def extract_transmission_info(transmission_desc: str) -> dict:
    if not isinstance(transmission_desc, str):
        return None
    transmission_desc = transmission_desc.lower()
    patterns = {
        "transmission_speed": r"(\d+)[\s-]speed",
        "auto": r"automatic|cvt|a/t|\sat|transmission overdrive switch",
        "manual": r"manual|m/t|\smt",
        "single_speed": r"single-speed"
    }
    transmission_info = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, transmission_desc)
        if key == "transmission_speed":
            transmission_info[key] = int(match.group(1)) if match else None
        elif key == "auto" and match:
            transmission_info["transmission_type"] = "automatic"
            break
        elif key == "manual" and match:
            transmission_info["transmission_type"] = "manual"
            break
        elif key == "single_speed" and match:
            transmission_info["transmission_type"] = "single_speed"
            break

    if "transmission_type" not in transmission_info:
        transmission_info["transmission_type"] = "automatic" if transmission_desc == "f" else "Unknown"

    return transmission_info

In [37]:
high_resale_price_brands = ['Mercedes-Benz', 'Bentley', 'Aston', 'Jaguar', 'Tesla', 'Lamborghini', 'Land', 'RAM', 
                            'Cadillac', 'Alfa', 'Ferrari', 'Porsche', 'Bugatti', 'McLaren', 'Rolls-Royce', 'Lucid', 
                            'Maserati', 'Rivian', 'Genesis']

In [38]:
# apply the "extract_engine_info" function to the 'engine' and 'brand' columns and extract each of the returned values into new columns
df_train[['horsepower', 'capacity', 'cylinders', 'fuel', 'turbo', 'hybrid']] = \
    df_train.apply(lambda x: extract_engine_info(x['engine'], x['brand']), axis=1).apply(pd.Series)
df_test[['horsepower', 'capacity', 'cylinders', 'fuel', 'turbo', 'hybrid']] = \
    df_test.apply(lambda x: extract_engine_info(x['engine'], x['brand']), axis=1).apply(pd.Series)
# apply the "extract_transmission_info" function to the 'transmission' column and extract each of the returned values into new columns
df_train[['transmission_speed', 'transmission_type']] = \
    df_train['transmission'].apply(extract_transmission_info).apply(pd.Series)
df_test[['transmission_speed', 'transmission_type']] = \
    df_test['transmission'].apply(extract_transmission_info).apply(pd.Series)
# by subtract model_year from current year create a new feature "age"
df_train['age'] = 2024 - df_train['model_year']
df_test['age'] = 2024 - df_test['model_year']
# Create the new column 'is_high_resale_price_brand'
df_train['is_high_resale_price_brand'] = df_train['brand'].isin(high_resale_price_brands)
df_test['is_high_resale_price_brand'] = df_test['brand'].isin(high_resale_price_brands)
# count missing values in each new column
na_counts = df_train[['cylinders', 'fuel', 'horsepower', 'capacity', 'transmission_speed', 'transmission_type', 'age']].isna().sum()
for col, count in na_counts.items():
    print(f'{col}_na_count = {count}')

cylinders_na_count = 22385
fuel_na_count = 8909
horsepower_na_count = 34067
capacity_na_count = 6915
transmission_speed_na_count = 86228
transmission_type_na_count = 0
age_na_count = 0


In [49]:
# Derived features
# Bin age into categories (e.g., 0-2, 3-5, 6-10) to capture non-linear relationships.
df_train['age_group'] = pd.cut(df_train['age'], bins=[0, 2, 5, 10], labels=['0-2', '3-5', '6-10'])
# Calculate mileage per year (milage / age) to account for usage.
df_train['mileage_per_year'] = df_train['milage'] / df_train['age']
# Mileage could be grouped into ranges (e.g., low, medium, high). This could help capture wear on the car 
# and relate it to depreciation in price.
df_train["milage_band"] = (df_train.milage//10_000)*10_000
df_train['power_to_capacity'] = df_train['horsepower'] / df_train['capacity']

array([17, 22,  7,  3,  6,  8,  4,  9, 13, 11,  1,  5, 12, 10, 16, 15,  2,
       21, 19, 23, 18, 24, 14, 20, 27, 26, 25, 30, 31, 28, 29,  0, 50, 32])

#### Handle Missing Values

In [42]:
def extract_fuel_type(engine_desc: str) -> str:    
    fuel_type_pattern = r'(Gasoline|Diesel|Flex|Electric|DOHC|OHV|SOHC|Dual Motor)'
    # Extract fuel type
    fuel_type_match = re.search(fuel_type_pattern, engine_desc)
    fuel_type = fuel_type_match.group(0) if fuel_type_match else None
    if fuel_type in ('DOHC', 'OHV', 'SOHC'):
        fuel_type = "Gasoline"
    elif fuel_type == 'Dual Motor':
        fuel_type = "Electric"
    return fuel_type

In [43]:
# if fuel_type isna and fuel column is not na, use fuel column to fill in fuel_type in df_train
df_train['fuel_type'] = df_train.apply(lambda row: row['fuel'] if pd.isnull(row['fuel_type']) else row['fuel_type'], axis=1)

In [44]:
# df_train_fueltype_na = df_train[df_train.fuel_type.isna()]
# df_test_fueltype_na = df_test[df_test.fuel_type.isna()]
# df_train_fueltype_na.loc[:, "fuel_type"] = df_train_fueltype_na["engine"].apply(extract_fuel_type)
# df_test_fueltype_na.loc[:, "fuel_type"] = df_test_fueltype_na["engine"].apply(extract_fuel_type)
# # update missing fuel_type in df_train from df_fueltype_na by joining on id
# df_train = df_train.merge(df_train_fueltype_na[["id", "fuel_type"]], on="id", how="left")
# # drop fuel_type column from df_train_fueltype_na
# df_train.drop("fuel_type_y", axis=1, inplace=True)
# df_train.rename(columns={"fuel_type_x": "fuel_type"}, inplace=True)
# df_test = df_test.merge(df_test_fueltype_na[["id", "fuel_type"]], on="id", how="left")
# df_test.drop("fuel_type_y", axis=1, inplace=True)
# df_test.rename(columns={"fuel_type_x": "fuel_type"}, inplace=True)

In [45]:
# impute "clean_title", "accident", "fuel_type" columns missing values with "Unknown"
df_train.fillna({"clean_title": "Unknown", "accident": "Unknown", "fuel_type": "Unknown", "fuel": "Unknown"}, inplace=True)
df_test.fillna({"clean_title": "Unknown", "accident": "Unknown", "fuel_type": "Unknown", "fuel": "Unknown"}, inplace=True)

In [46]:
df_train.to_csv(DATA_WRITEPATH + "train_withorig_preprocessed.csv", index=False)
df_test.to_csv(DATA_WRITEPATH + "test_withorig_preprocessed.csv", index=False)
df_train.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,...,horsepower,capacity,cylinders,fuel,turbo,hybrid,transmission_speed,transmission_type,age,is_high_resale_price_brand
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,...,172.0,1.6,4.0,gasoline,False,False,NaN,automatic,17,False
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,...,252.0,3.9,8.0,gasoline,False,False,NaN,automatic,22,False
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,...,320.0,5.3,8.0,flex,False,False,NaN,automatic,22,False
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,...,420.0,5.0,8.0,gasoline,False,False,NaN,Unknown,7,True
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,...,208.0,2.0,4.0,gasoline,False,False,7.0,automatic,3,True


In [47]:
len(df_train)

192542

In [48]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192542 entries, 0 to 192541
Data columns (total 22 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   brand                       192542 non-null  object 
 1   model                       192542 non-null  object 
 2   model_year                  192542 non-null  int64  
 3   milage                      192542 non-null  int64  
 4   fuel_type                   192542 non-null  object 
 5   engine                      192542 non-null  object 
 6   transmission                192542 non-null  object 
 7   ext_col                     192542 non-null  object 
 8   int_col                     192542 non-null  object 
 9   accident                    192542 non-null  object 
 10  clean_title                 192542 non-null  object 
 11  price                       192542 non-null  int64  
 12  horsepower                  158475 non-null  float64
 13  capacity      